In [67]:
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import numpy as np
import seaborn as sns
from pandas.plotting import register_matplotlib_converters

from statsmodels.tsa.seasonal import MSTL
from statsmodels.tsa.seasonal import DecomposeResult
from tqdm import tqdm

In [68]:
stores_data  = pd.read_excel('ProjectStores.xlsx')
stores_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Store       25 non-null     object
 1   City        25 non-null     object
 2   station     25 non-null     object
 3   Is College  25 non-null     int64 
 4   Number      25 non-null     int64 
 5   id          25 non-null     int64 
dtypes: int64(3), object(3)
memory usage: 1.3+ KB


In [69]:
weather_data = pd.read_csv('city_weather_data.csv')
seasonality_data= pd.read_csv('sales_seasonality.csv')

In [70]:
weather_data['date'] = pd.to_datetime(weather_data['date'])
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6537 entries, 0 to 6536
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   station  6537 non-null   object        
 1   date     6537 non-null   datetime64[ns]
 2   PRCP     6525 non-null   float64       
 3   SNOW     6153 non-null   float64       
 4   TMAX     6510 non-null   float64       
 5   TMIN     6510 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 306.6+ KB


In [71]:
seasonality_data['Date'] = pd.to_datetime(seasonality_data['Date'])
seasonality_data['Store_ID'] = seasonality_data['Store_ID'].astype(str).str.replace(',', '').astype(int)
seasonality_data.info()
seasonality_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25057 entries, 0 to 25056
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Date                25057 non-null  datetime64[ns]
 1   Annual_Seasonality  24115 non-null  float64       
 2   Weekly_Seasonality  25057 non-null  float64       
 3   Store_ID            25057 non-null  int32         
dtypes: datetime64[ns](1), float64(2), int32(1)
memory usage: 685.3 KB


,Date,Annual_Seasonality,Weekly_Seasonality,Store_ID
0,2022-03-23,48.442202,134.480549,14
1,2022-03-24,152.780204,129.593819,14
2,2022-03-25,322.173576,213.914156,14
3,2022-03-26,50.232197,-324.036395,14
4,2022-03-27,144.313304,-234.285401,14


In [72]:
data = weather_data.merge(stores_data, how='left', left_on='station', right_on='station')

In [73]:
data.info()
data.head()
data.to_csv("temp.csv",index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27380 entries, 0 to 27379
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   station     27380 non-null  object        
 1   date        27380 non-null  datetime64[ns]
 2   PRCP        27368 non-null  float64       
 3   SNOW        26980 non-null  float64       
 4   TMAX        27353 non-null  float64       
 5   TMIN        27353 non-null  float64       
 6   Store       27380 non-null  object        
 7   City        27380 non-null  object        
 8   Is College  27380 non-null  int64         
 9   Number      27380 non-null  int64         
 10  id          27380 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(3), object(3)
memory usage: 2.3+ MB


In [105]:
final_data = (data.copy()).merge(seasonality_data, how='left', left_on=['id','date'], right_on=['Store_ID','Date'])
final_data = final_data.groupby('station').apply(lambda x: x.sort_values('date')).reset_index(drop=True)

C:\Users\mmcdo\AppData\Local\Temp\ipykernel_4012\3097771553.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  final_data = final_data.groupby('station',  as_index=False).apply(lambda x: x.sort_values('date')).reset_index(drop=True)


In [106]:
final_data

,station,date,PRCP,SNOW,TMAX,TMIN,Store,City,Is College,Number,id,Date,Annual_Seasonality,Weekly_Seasonality,Store_ID
0,AA,2022-03-01,0.000000,0.00000,35.96,19.94,Ann Arbor,Ann Arbor,1,11,35,NaT,NaN,NaN,NaN
1,AA,2022-03-02,0.000000,0.00000,50.00,21.02,Ann Arbor,Ann Arbor,1,11,35,NaT,NaN,NaN,NaN
2,AA,2022-03-03,0.393701,0.11811,42.98,17.96,Ann Arbor,Ann Arbor,1,11,35,NaT,NaN,NaN,NaN
3,AA,2022-03-04,0.000000,0.00000,32.00,14.00,Ann Arbor,Ann Arbor,1,11,35,NaT,NaN,NaN,NaN
4,AA,2022-03-05,0.000000,0.00000,39.92,15.98,Ann Arbor,Ann Arbor,1,11,35,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27375,STL,2025-02-28,0.000000,0.00000,75.02,37.94,Downtown St. Louis,St Louis,0,101,1150,2025-02-28,480.971042,527.632451,1150.0
27376,STL,2025-03-01,0.000000,0.00000,48.92,28.22,"Delmar Loop, St. Louis",St Louis,0,59,1106,2025-03-01,339.491416,1310.840702,1106.0
27377,STL,2025-03-01,0.000000,0.00000,48.92,28.22,Downtown St. Louis,St Louis,0,101,1150,2025-03-01,724.219425,1186.026733,1150.0
27378,STL,2025-03-01,0.000000,0.00000,48.92,28.22,"Central West End, St. Louis",St Louis,0,55,1101,2025-03-01,343.418347,962.101391,1101.0


In [107]:

final_data[['TMAX',"TMIN","SNOW","PRCP"]] = final_data[['TMAX',"TMIN","SNOW","PRCP"]].ffill()

In [108]:
final_data.round(2).to_csv("weather_seasonality.csv",index=False)

In [66]:
seasonality_data[seasonality_data['Store_ID']==2052]

,Date,Annual_Seasonality,Weekly_Seasonality,Store_ID
23019,2022-03-23,281.397133,-271.460843,2052
23020,2022-03-24,554.587342,-335.051622,2052
23021,2022-03-25,293.003871,826.488401,2052
23022,2022-03-26,51.449312,441.956554,2052
23023,2022-03-27,-85.721055,392.845028,2052
...,...,...,...,...
48225,2025-03-18,78.061961,-382.352413,2052
48226,2025-03-19,232.585702,-404.972229,2052
48227,2025-03-20,-117.547310,-135.122918,2052
48228,2025-03-21,-323.453482,659.938125,2052
